In [10]:
import pandas as pd
from scipy.stats import chi2_contingency

# Carregar o arquivo CSV
df = pd.read_csv('pns2019.csv')

# Selecionar colunas específicas
colunas_selecionadas = ['V0001', 'Q06306']
dornocoracao = df[colunas_selecionadas]

# Substituir strings que representam NaN por NaN do pandas
dornocoracao['Q06306'] = pd.to_numeric(dornocoracao['Q06306'], errors='coerce')

# Remover linhas onde Q06306 é NaN
dornocoracao_limpo = dornocoracao.dropna(subset=['Q06306'])

# Mapear códigos de estado para nomes
mapeamento_estados = {
    11: 'Rondônia',
    12: 'Acre',
    13: 'Amazonas',
    14: 'Roraima',
    15: 'Pará',
    16: 'Amapá',
    17: 'Tocantins',
    21: 'Maranhão',
    22: 'Piauí',
    23: 'Ceará',
    24: 'Rio Grande do Norte',
    25: 'Paraíba',
    26: 'Pernambuco',
    27: 'Alagoas',
    28: 'Sergipe',
    29: 'Bahia',
    31: 'Minas Gerais',
    32: 'Espírito Santo',
    33: 'Rio de Janeiro',
    35: 'São Paulo',
    41: 'Paraná',
    42: 'Santa Catarina',
    43: 'Rio Grande do Sul',
    50: 'Mato Grosso do Sul',
    51: 'Mato Grosso',
    52: 'Goiás',
    53: 'Distrito Federal'
}

# Mapear estados para regiões
mapeamento_regioes = {
    'Norte': ['Rondônia', 'Acre', 'Amazonas', 'Roraima', 'Pará', 'Amapá', 'Tocantins'],
    'Nordeste': ['Maranhão', 'Piauí', 'Ceará', 'Rio Grande do Norte', 'Paraíba', 'Pernambuco', 'Alagoas', 'Sergipe', 'Bahia'],
    'Sudeste': ['Minas Gerais', 'Espírito Santo', 'Rio de Janeiro', 'São Paulo'],
    'Sul': ['Paraná', 'Santa Catarina', 'Rio Grande do Sul'],
    'Centro-Oeste': ['Mato Grosso do Sul', 'Mato Grosso', 'Goiás', 'Distrito Federal']
}

# Substituir os códigos pelos nomes dos estados
dornocoracao_limpo['V0001'] = dornocoracao_limpo['V0001'].map(mapeamento_estados)

# Mapear estados para regiões
def estado_para_regiao(estado):
    for regiao, estados in mapeamento_regioes.items():
        if estado in estados:
            return regiao
    return None

dornocoracao_limpo['Região'] = dornocoracao_limpo['V0001'].apply(estado_para_regiao)

# Criar uma tabela de contingência para doença por região
contagem_doenca = pd.crosstab(dornocoracao_limpo['Região'], dornocoracao_limpo['Q06306'])

# Lista de pares de regiões para teste
pares_regioes = [
    ('Centro-Oeste', 'Nordeste'),
    ('Centro-Oeste', 'Norte'),
    ('Centro-Oeste', 'Sudeste'),
    ('Centro-Oeste', 'Sul'),
    ('Nordeste', 'Norte'),
    ('Nordeste', 'Sudeste'),
    ('Nordeste', 'Sul'),
    ('Norte', 'Sudeste'),
    ('Norte', 'Sul'),
    ('Sudeste', 'Sul')
]

# Valor crítico para alfa = 0.1% (13.82)
valor_critico = 13.82

# Função para verificar valores esperados
def verificar_valores_esperados(tabela):
    chi2, p, dof, expected = chi2_contingency(tabela)
    print("Valores esperados:")
    print(expected)
    return chi2, p, expected

# Realizar o teste de qui-quadrado para cada par de regiões
for regiao1, regiao2 in pares_regioes:
    # Criar a tabela de contingência para o par de regiões
    tabela_contingencia = contagem_doenca.loc[[regiao1, regiao2]]
    
    # Verificar valores esperados
    print(f"\nVerificando valores esperados para {regiao1} e {regiao2}:")
    chi2, p, expected = verificar_valores_esperados(tabela_contingencia)
    
    # Verificar se todos os valores esperados são maiores que 5
    if (expected < 5).any():
        print(f"Atenção: Alguns valores esperados são menores que 5. O teste de qui-quadrado pode não ser confiável para {regiao1} e {regiao2}.")
    else:
        # Exibir os resultados
        print(f"Teste de qui-quadrado entre {regiao1} e {regiao2}:")
        print(f"Chi2: {chi2}, p-value: {p}")
        
        # Avaliar se o teste é aceito ou rejeitado
        if chi2 > valor_critico:
            print(f"Conclusão: Rejeitar H0 (Chi2 > {valor_critico})")
        else:
            print(f"Conclusão: Aceitar H0 (Chi2 <= {valor_critico})")
        print("----------------------------------------")

C:\Users\llays\AppData\Local\Temp\ipykernel_6160\3975181220.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dornocoracao['Q06306'] = pd.to_numeric(dornocoracao['Q06306'], errors='coerce')
C:\Users\llays\AppData\Local\Temp\ipykernel_6160\3975181220.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dornocoracao_limpo['V0001'] = dornocoracao_limpo['V0001'].map(mapeamento_estados)
C:\Users\llays\AppData\Local\Temp\ipykernel_6160\3975181220.py:67: SettingWithCopyWarning: 
A value is trying to be set on a


Verificando valores esperados para Centro-Oeste e Nordeste:
Valores esperados:
[[  494.09217491  9903.90782509]
 [ 1498.90782509 30045.09217491]]
Teste de qui-quadrado entre Centro-Oeste e Nordeste:
Chi2: 9.971372162646956, p-value: 0.0015899295743501472
Conclusão: Aceitar H0 (Chi2 <= 13.82)
----------------------------------------

Verificando valores esperados para Centro-Oeste e Norte:
Valores esperados:
[[  439.68685714  9958.31314286]
 [  744.31314286 16857.68685714]]
Teste de qui-quadrado entre Centro-Oeste e Norte:
Chi2: 48.93294984802163, p-value: 2.6486367020783273e-12
Conclusão: Rejeitar H0 (Chi2 > 13.82)
----------------------------------------

Verificando valores esperados para Centro-Oeste e Sudeste:
Valores esperados:
[[  631.21377531  9766.78622469]
 [ 1203.78622469 18626.21377531]]
Teste de qui-quadrado entre Centro-Oeste e Sudeste:
Chi2: 15.130562528011362, p-value: 0.00010032588998570418
Conclusão: Rejeitar H0 (Chi2 > 13.82)
----------------------------------------
